<a href="https://colab.research.google.com/github/ADRopentech/International_Trade_Network_Analysis/blob/main/Code/Network_Analysis_Rice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title intalling packages

!sudo apt-get install graphviz graphviz-dev
!pip install pygraphviz

!pip install --upgrade comtradeapicall
!pip install dash-cytoscape==0.2.0

!pip install dash-bootstrap-components

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.42.2-6).
libgraphviz-dev is already the newest version (2.42.2-6).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
# @title import library
import pandas as pd
import numpy as np
import math

import comtradeapicall
import time

import pygraphviz
import networkx as nx
import dash_cytoscape as cyto
cyto.load_extra_layouts()
from plotly.colors import hex_to_rgb

from dash import Dash, html, Input, Output, callback, ctx
import dash_bootstrap_components as dbc
from datetime import datetime;
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# @title import files
url_hs_codes = "https://raw.githubusercontent.com/ADRopentech/Visualizing-Global-Trade-Networks/main/comtrade_codes/harmonized-system.csv"
url_partner_codes = "https://raw.githubusercontent.com/ADRopentech/Visualizing-Global-Trade-Networks/main/comtrade_codes/partnerAreas.csv"
url_reporter_codes = "https://raw.githubusercontent.com/ADRopentech/Visualizing-Global-Trade-Networks/main/comtrade_codes/reporterAreas.csv"

commodities = pd.read_csv(url_hs_codes)
partners = pd.read_csv(url_partner_codes)
reporters = pd.read_csv(url_reporter_codes)

In [ ]:
# @title commtrade subscription key

subscription_key = "18f9bd542c7e439a8b478e4a59807a60"

In [ ]:
comtrade_exp = comtradeapicall.getFinalData(
    subscription_key,
    typeCode='C',                                   # C = commodities
    freqCode='A',                                   # A = annual
    clCode='HS',
    period='2022',                                  # year; several years should be listed in order and separated by commas
    reporterCode=None,                              # all exporters
    cmdCode= '1006',                                # wheat and meslin hs-code; several codes should be separated by commas
    flowCode='X',                                   # X = export
    partnerCode=None,                               # all importers
    partner2Code='0',
    customsCode='C00',
    motCode='0',
    maxRecords=250000)

time.sleep(10)

comtrade_imp = comtradeapicall.getFinalData(
    subscription_key,
    typeCode='C',
    freqCode='A',
    clCode='HS',
    period='2022',
    reporterCode=None,                              # all importers
    cmdCode='1006',
    flowCode='M',                                   # M = import
    partnerCode=None,                               # all exporters
    partner2Code='0',
    customsCode='C00',
    motCode='0',
    maxRecords=250000)

comtrade_imp = comtrade_imp[
    comtrade_imp['reporterCode'] != comtrade_imp['partnerCode']]  # Getting rid of a country's trade with itself

**Data Processing**

In [ ]:
# @title creating Total exports/imports and exports/imports by country dataset
export_totals = comtrade_exp[comtrade_exp['partnerCode'] == 0][[             # Total exports by country
    'reporterCode', 'partnerCode', 'fobvalue'
]].groupby(['reporterCode', 'partnerCode']).agg('sum').reset_index()

import_totals = comtrade_imp[comtrade_imp['partnerCode'] == 0][[             # Total imports by country
    'reporterCode', 'partnerCode', 'cifvalue'
]].groupby(['reporterCode', 'partnerCode']).agg('sum').reset_index()

export_by_country = comtrade_exp[comtrade_exp['partnerCode'] != 0][[         # Exports by exporter & importer
    'reporterCode', 'partnerCode', 'fobvalue'
]].groupby(['reporterCode', 'partnerCode']).agg('sum').reset_index()

import_by_country = comtrade_imp[comtrade_imp['partnerCode'] != 0][[         # Imports by importer & exporter
    'reporterCode', 'partnerCode', 'cifvalue'
]].groupby(['reporterCode', 'partnerCode']).agg('sum').reset_index()

In [ ]:
# @title appending 0s to make exporter and inporter code uniform
for dataset in [export_totals, import_totals, export_by_country, import_by_country]:
    for col in ['reporterCode', 'partnerCode']:
        dataset[col] = [
            '00' + str(x) if len(str(x)) == 1
            else '0' +  str(x) if len(str(x)) == 2
            else str(x)
            for x in dataset[col].tolist()
        ]

In [ ]:
# @title changing names of the columns
export_by_country.columns = ['exporter', 'importer', 'value']
import_by_country.columns = ['importer', 'exporter', 'value']

In [ ]:
# @title creating data set for total trade reported by countries
total_trade = export_totals[[
    'reporterCode',
    'fobvalue'
]].set_index('reporterCode').join(
    import_totals[['reporterCode', 'cifvalue']].set_index('reporterCode'),
    how='outer').reset_index().rename(columns={
        'reporterCode': 'country_code',
        'fobvalue': 'export',
        'cifvalue': 'import'
    }).set_index('country_code')

In [ ]:
# @title Marking the rows with missing official exports or imports data
for col in ['export', 'import']:
    total_trade[col + '_note'] = [1 if math.isnan(x) else 0 for x in total_trade[col].tolist()]

In [ ]:
display(total_trade)

,export,import,export_note,import_note
country_code,,,,
008,NaN,6.951423e+04,1,0
020,NaN,7.541966e+05,1,0
024,1.687723e+05,3.704230e+08,0,0
028,NaN,1.969455e+06,1,0
031,3.221069e+04,4.255051e+07,0,0
...,...,...,...,...
842,1.703807e+09,1.405742e+09,0,0
854,7.271550e+04,1.438515e+08,0,0
858,4.945814e+08,7.715741e+05,0,0


In [ ]:
# @title countries exports according to its importers
imports_to_add = export_by_country[['exporter', 'importer', 'value']].groupby('importer').agg('sum')
print(imports_to_add)

                                                   exporter         value
importer                                                                 
004                                   156360398586699784792  1.787991e+08
008       0401002763003804104905285866206997247647847928...  1.540441e+07
010                                                  710826  1.798326e+04
012                          076380586620699704724764792858  8.945726e+07
016                                      144242404490554764  2.331709e+05
...                                                     ...           ...
876                                         156242251554764  1.543227e+05
882                                            156242490554  4.068753e+06
887                             048410512586699710764784826  4.443455e+08
894                          072144454516586699710764784834  2.255296e+06
899       0360440761001882422512763083723844905045285336...  2.213155e+08

[229 rows x 2 columns]


In [ ]:
# @title countries imports according to its exporters
exports_to_add = import_by_country[['importer', 'exporter', 'value']].groupby('exporter').agg('sum')
print(exports_to_add)

                                                   importer        value
exporter                                                                
004                                               040554860     8048.367
008                                            024499646842    34558.334
012                                                  251288     1718.496
020                                               144251646     2086.814
024                                                  180516    73364.651
...                                                     ...          ...
860                          031040124398410417462762784842   385299.572
862                                         040048442462842    26306.774
887                                                  124512    52239.272
894                                            072180516716   380157.388
899       0441001321521702042513083763844424995165336206...  3142692.600

[175 rows x 2 columns]


In [ ]:
# @title total trade to add
trade_to_add = exports_to_add.rename(columns = {'value':'export_by_partners'}).join(imports_to_add.rename(columns = {'value':'import_by_partners'}),how = 'outer')
print(trade_to_add)

                                              importer  export_by_partners  \
004                                          040554860            8048.367   
008                                       024499646842           34558.334   
010                                                NaN                 NaN   
012                                             251288            1718.496   
016                                                NaN                 NaN   
..                                                 ...                 ...   
876                                                NaN                 NaN   
882                                                NaN                 NaN   
887                                             124512           52239.272   
894                                       072180516716          380157.388   
899  0441001321521702042513083763844424995165336206...         3142692.600   

                                              exporter  import_

In [ ]:
display(total_trade)

,export,import,export_note,import_note
country_code,,,,
008,NaN,6.951423e+04,1,0
020,NaN,7.541966e+05,1,0
024,1.687723e+05,3.704230e+08,0,0
028,NaN,1.969455e+06,1,0
031,3.221069e+04,4.255051e+07,0,0
...,...,...,...,...
842,1.703807e+09,1.405742e+09,0,0
854,7.271550e+04,1.438515e+08,0,0
858,4.945814e+08,7.715741e+05,0,0


**Nodes**

In [ ]:
# @title creating nodes dataset
df_nodes = total_trade.join(trade_to_add, how='outer')[['export', 'import', 'export_by_partners', 'import_by_partners']]
df_nodes[['export', 'import', 'export_by_partners','import_by_partners']] = df_nodes[['export', 'import', 'export_by_partners', 'import_by_partners']].fillna(0)
display(df_nodes)

,export,import,export_by_partners,import_by_partners
004,0.000,0.000000e+00,8048.367,1.787991e+08
008,0.000,6.951423e+04,34558.334,1.540441e+07
010,0.000,0.000000e+00,0.000,1.798326e+04
012,0.000,0.000000e+00,1718.496,8.945726e+07
016,0.000,0.000000e+00,0.000,2.331709e+05
...,...,...,...,...
876,0.000,0.000000e+00,0.000,1.543227e+05
882,0.000,0.000000e+00,0.000,4.068753e+06
887,0.000,0.000000e+00,52239.272,4.443455e+08
894,3754980.584,1.537920e+07,380157.388,2.255296e+06


In [ ]:
# @title creating 'trade' column in nodes dataframe
df_nodes['trade'] = [
    ex + im if ex != 0 and im != 0
    else exp + im if ex == 0 and im != 0
    else ex + imp if ex != 0 and im == 0
    else exp + imp for ex, im, exp, imp in zip(df_nodes['export'], df_nodes['import'], df_nodes['export_by_partners'], df_nodes['import_by_partners'])
]
df_nodes = df_nodes[df_nodes['trade']>0]
display(df_nodes)

,export,import,export_by_partners,import_by_partners,trade
004,0.000,0.000000e+00,8048.367,1.787991e+08,1.788072e+08
008,0.000,6.951423e+04,34558.334,1.540441e+07,1.040726e+05
010,0.000,0.000000e+00,0.000,1.798326e+04,1.798326e+04
012,0.000,0.000000e+00,1718.496,8.945726e+07,8.945898e+07
016,0.000,0.000000e+00,0.000,2.331709e+05,2.331709e+05
...,...,...,...,...,...
876,0.000,0.000000e+00,0.000,1.543227e+05,1.543227e+05
882,0.000,0.000000e+00,0.000,4.068753e+06,4.068753e+06
887,0.000,0.000000e+00,52239.272,4.443455e+08,4.443978e+08
894,3754980.584,1.537920e+07,380157.388,2.255296e+06,1.913418e+07


In [ ]:
# @title rescaling and setting diameters of nodes
df_nodes['trade_rescaled'] = df_nodes['trade'] / df_nodes['trade'].max() * 60                            # Rescaling the sizes: max = 60 and min = 0.15
df_nodes['trade_rescaled'] = [ 0.15 if x <= 0.15 else x for x in df_nodes['trade_rescaled'].tolist()]
df_nodes['diameter'] = [np.sqrt(x / np.pi) * 2 for x in df_nodes['trade_rescaled'].tolist()]# Nodes' diameters

display(df_nodes)
diameter_dict = df_nodes['diameter'].to_dict()
print(diameter_dict)

,export,import,export_by_partners,import_by_partners,trade,trade_rescaled,diameter
004,0.000,0.000000e+00,8048.367,1.787991e+08,1.788072e+08,0.995766,1.125988
008,0.000,6.951423e+04,34558.334,1.540441e+07,1.040726e+05,0.150000,0.437019
010,0.000,0.000000e+00,0.000,1.798326e+04,1.798326e+04,0.150000,0.437019
012,0.000,0.000000e+00,1718.496,8.945726e+07,8.945898e+07,0.498192,0.796440
016,0.000,0.000000e+00,0.000,2.331709e+05,2.331709e+05,0.150000,0.437019
...,...,...,...,...,...,...,...
876,0.000,0.000000e+00,0.000,1.543227e+05,1.543227e+05,0.150000,0.437019
882,0.000,0.000000e+00,0.000,4.068753e+06,4.068753e+06,0.150000,0.437019
887,0.000,0.000000e+00,52239.272,4.443455e+08,4.443978e+08,2.474825,1.775118
894,3754980.584,1.537920e+07,380157.388,2.255296e+06,1.913418e+07,0.150000,0.437019


{'004': 1.1259881172913868, '008': 0.4370193722368316, '010': 0.4370193722368316, '012': 0.7964404573576455, '016': 0.4370193722368316, '020': 0.4370193722368316, '024': 1.6210232388156673, '028': 0.4370193722368316, '031': 0.5494878012955208, '032': 1.2041663195743297, '036': 1.8530658591639102, '040': 0.6712828480262913, '044': 0.4370193722368316, '048': 0.6911693510669231, '050': 1.6525432345996673, '051': 0.4370193722368316, '052': 0.4370193722368316, '056': 2.3471914545446606, '060': 0.4370193722368316, '064': 0.4370193722368316, '068': 0.4370193722368316, '070': 0.4370193722368316, '072': 0.5525555061967264, '076': 2.7047561930582265, '084': 0.4370193722368316, '086': 0.4370193722368316, '090': 0.4370193722368316, '092': 0.4370193722368316, '096': 0.4370193722368316, '100': 1.0914107132898208, '104': 2.3623687457878386, '108': 0.4370193722368316, '112': 0.4370193722368316, '116': 1.7246302100092885, '120': 1.4740832544034475, '124': 1.8824852263881238, '132': 0.4370193722368316, 

In [ ]:
# Keep only countries with non-zero trade
export_by_country = export_by_country[(export_by_country['exporter'].isin(df_nodes.index.tolist())) & (export_by_country['importer'].isin(df_nodes.index.tolist()))]
import_by_country = import_by_country[(import_by_country['exporter'].isin(df_nodes.index.tolist())) & (import_by_country['importer'].isin(df_nodes.index.tolist()))]

df_list = []

for dataset in [export_by_country, import_by_country]:
  dataset['source_target'] = ['_'.join(sorted([exporter, importer]))
  for exporter, importer in zip(dataset['exporter'], dataset['importer'])
  ]

  dataset['order'] = dataset.groupby('source_target').cumcount()

  df = dataset[dataset['order']==0].set_index('source_target')[[
      'exporter', 'importer', 'value'
  ]].rename(columns = {
      'value' : 'to'
  }).join(dataset[dataset['order']==1].set_index('source_target')[['value']].rename(columns={'value':'back'}))

  df_list.append(df)



[              exporter importer            to      back
 source_target                                          
 024_180            024      180  1.600740e+05       NaN
 024_192            024      192  3.245990e+02       NaN
 024_196            024      196  4.526100e+01       NaN
 024_470            024      470  1.173943e+03       NaN
 024_516            024      516  1.421504e+03  2706.125
 ...                ...      ...           ...       ...
 757_858            858      757  3.341530e+05       NaN
 826_858            858      826  2.311455e+07       NaN
 858_862            858      862  5.300186e+07       NaN
 180_894            894      180  3.753803e+06       NaN
 729_894            894      729  4.723200e+01       NaN
 
 [2856 rows x 4 columns],
               exporter importer         to    back
 source_target                                     
 008_276            276      008  11757.653     NaN
 008_300            300      008  57756.575     NaN
 020_251            251

In [ ]:
source_target_df = df_list[0].combine_first(df_list[1])
source_target_df = source_target_df.fillna(0)

source_target_df[['source', 'target']] = [[exporter, importer] if to - back >= 0 else [importer, exporter]
                                          for exporter, importer, to, back in
                                          zip(source_target_df['exporter'], source_target_df['importer'],
                                              source_target_df['to'], source_target_df['back'])
                                          ]

display(source_target_df)

,exporter,importer,to,back,source,target
source_target,,,,,,
004_040,004,040,4.950382e+03,0.0,004,040
004_156,156,004,2.152815e+06,0.0,156,004
004_360,360,004,2.824300e+04,0.0,360,004
004_398,398,004,6.684600e+02,0.0,398,004
004_554,004,554,1.865985e+03,0.0,004,554
...,...,...,...,...,...,...
842_860,860,842,1.666940e+05,43590.0,860,842
842_862,842,862,8.647190e+05,0.0,842,862
842_894,842,894,3.781557e+04,0.0,842,894


In [ ]:
# @title links
df_links = source_target_df.reset_index()[['source','target']]

display(df_links)

,source,target
0,004,040
1,156,004
2,360,004
3,398,004
4,004,554
...,...,...
3962,860,842
3963,842,862
3964,842,894
3965,858,862


In [ ]:
display(export_by_country)
display(import_by_country)

,exporter,importer,value,source_target,order
0,024,180,160073.956,024_180,0
1,024,192,324.599,024_192,0
2,024,196,45.261,024_196,0
3,024,470,1173.943,024_470,0
4,024,516,1421.504,024_516,0
...,...,...,...,...,...
3389,860,398,3130.000,398_860,1
3390,894,072,23.616,072_894,1
3391,894,180,3753802.731,180_894,0
3392,894,516,1107.005,516_894,1


,importer,exporter,value,source_target,order
0,008,276,11757.653,008_276,0
1,008,300,57756.575,008_300,0
2,020,251,18192.857,020_251,0
3,020,380,691.954,020_380,0
4,020,528,5057.063,020_528,0
...,...,...,...,...,...
3505,894,800,313.463,800_894,0
3506,894,826,914.544,826_894,0
3507,894,834,20948.832,834_894,0
3508,894,842,37815.571,842_894,0


In [ ]:
# @title Ranking trade partners for each country depending on the total cost of the commodity delivered

rank_df = export_by_country[['importer', 'exporter', 'value']].set_index([
    'importer', 'exporter'
]).combine_first(import_by_country[[
    'importer', 'exporter', 'value'
    ]].set_index(['importer','exporter'])).reset_index()

display(rank_df)

,importer,exporter,value
0,004,156,2.152815e+06
1,004,360,2.824300e+04
2,004,398,6.684600e+02
3,004,586,1.764640e+08
4,004,699,1.741201e+04
...,...,...,...
4841,899,604,8.306030e+03
4842,899,699,1.506575e+05
4843,899,792,1.584713e+06
4844,899,807,5.444100e+02


In [ ]:
rank_df['supplier_rank'] = rank_df.sort_values(by = 'value', ascending = False).groupby('importer').cumcount() + 1

display(rank_df)

supplier_rank_dict = rank_df.set_index(['importer', 'exporter'])['supplier_rank'].to_dict()

print(supplier_rank_dict)

,importer,exporter,value,supplier_rank
0,004,156,2.152815e+06,2
1,004,360,2.824300e+04,5
2,004,398,6.684600e+02,7
3,004,586,1.764640e+08,1
4,004,699,1.741201e+04,6
...,...,...,...,...
4841,899,604,8.306030e+03,12
4842,899,699,1.506575e+05,4
4843,899,792,1.584713e+06,3
4844,899,807,5.444100e+02,16


{('004', '156'): 2, ('004', '360'): 5, ('004', '398'): 7, ('004', '586'): 1, ('004', '699'): 6, ('004', '784'): 3, ('004', '792'): 4, ('008', '040'): 13, ('008', '100'): 3, ('008', '276'): 12, ('008', '300'): 6, ('008', '380'): 5, ('008', '410'): 17, ('008', '490'): 8, ('008', '528'): 16, ('008', '586'): 2, ('008', '620'): 9, ('008', '699'): 10, ('008', '724'): 4, ('008', '764'): 11, ('008', '784'): 15, ('008', '792'): 1, ('008', '807'): 7, ('008', '842'): 14, ('010', '710'): 2, ('010', '826'): 1, ('012', '076'): 3, ('012', '380'): 10, ('012', '586'): 5, ('012', '620'): 9, ('012', '699'): 1, ('012', '704'): 8, ('012', '724'): 7, ('012', '764'): 2, ('012', '792'): 4, ('012', '858'): 6, ('016', '144'): 5, ('016', '242'): 3, ('016', '404'): 6, ('016', '490'): 1, ('016', '554'): 4, ('016', '764'): 2, ('020', '251'): 2, ('020', '380'): 5, ('020', '528'): 3, ('020', '620'): 4, ('020', '699'): 6, ('020', '724'): 1, ('024', '008'): 22, ('024', '032'): 10, ('024', '076'): 5, ('024', '104'): 3, 

In [ ]:
df_links['supplier_rank_source'] = df_links.set_index(['target', 'source']).index.map(supplier_rank_dict)

df_links['supplier_rank_target'] = df_links.set_index(['source', 'target']).index.map(supplier_rank_dict)

display(df_links)

,source,target,supplier_rank_source,supplier_rank_target
0,004,040,37,NaN
1,156,004,2,NaN
2,360,004,5,NaN
3,398,004,7,NaN
4,004,554,26,NaN
...,...,...,...,...
3962,860,842,34,5.0
3963,842,862,6,44.0
3964,842,894,9,NaN
3965,858,862,3,NaN


In [ ]:
link_dict = dict()

for e in export_by_country.exporter:
  link_dict[e] = []
for i in export_by_country.importer:
  if i not in link_dict.keys():
    link_dict[i] = []
for i in import_by_country.importer:
  if i not in link_dict.keys():
    link_dict[i] = []
for e in import_by_country.exporter:
  if e not in link_dict.keys():
    link_dict[e] = []

for e in export_by_country.exporter:
    data = export_by_country[export_by_country.exporter == e]
    importers_list = data['importer'].unique().tolist()
    for i in importers_list:
        link_dict[e].append(i)

for i in export_by_country.importer:
    data = export_by_country[export_by_country.importer == i]
    exporters_list = data['exporter'].unique().tolist()
    for e in exporters_list:
        link_dict[i].append(e)

for e in import_by_country.exporter:
    data = import_by_country[import_by_country.exporter == e]
    importers_list = data['importer'].unique().tolist()
    for i in importers_list:
        link_dict[e].append(i)

for i in import_by_country.importer:
    data = import_by_country[import_by_country.importer == i]
    exporters_list = data['exporter'].unique().tolist()
    for e in exporters_list:
        link_dict[i].append(e)

for key in link_dict.keys():
    link_dict[key] = len(list(dict.fromkeys(link_dict[key])))

print(link_dict)

{'024': 32, '031': 27, '032': 50, '036': 71, '040': 72, '044': 26, '048': 52, '051': 20, '052': 17, '056': 115, '060': 9, '070': 25, '072': 15, '076': 112, '084': 12, '096': 20, '100': 63, '104': 68, '108': 18, '116': 79, '124': 108, '144': 57, '152': 30, '156': 150, '170': 36, '180': 31, '188': 23, '191': 36, '196': 44, '203': 57, '204': 30, '208': 66, '214': 21, '218': 22, '222': 17, '231': 30, '233': 41, '242': 28, '246': 56, '251': 133, '268': 27, '275': 22, '276': 109, '288': 70, '300': 70, '308': 12, '320': 24, '328': 47, '344': 44, '348': 43, '352': 41, '360': 53, '372': 68, '376': 44, '380': 150, '384': 37, '388': 17, '392': 100, '398': 37, '400': 40, '404': 52, '410': 88, '414': 39, '417': 15, '422': 76, '426': 3, '428': 33, '440': 42, '442': 52, '450': 22, '454': 25, '458': 41, '462': 45, '470': 35, '480': 28, '484': 29, '490': 81, '496': 21, '498': 39, '499': 31, '504': 30, '508': 43, '512': 41, '516': 30, '524': 21, '528': 127, '533': 11, '554': 54, '558': 14, '566': 37, '5